아래 데이터를 RDD로 만들고, 성적의 합계 및 평균을 계산하세요.

문항별로 RDD를 만들지 말고, 하나의 RDD만을 생성하고, 이를 변형해서 문제를 푸세요.



이름 | 과목 | 점수

-----|-----|-----

김하나 | English | 100

김하나 | Math | 80

임하나 | English | 70

임하나 | Math | 100

김갑돌 | English | 82.3

김갑돌 | Math | 98.5

In [1]:
import os
import pyspark

myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
recordList = [('김하나',('English', 100)), ('김하나', ('Math', 80)), ('임하나', ('English', 70)), ('임하나', ('Math', 100)), 
         ('김갑돌', ('English', 82.3)), ('김갑돌', ('Math', 98.5))]

recordRdd=spark.sparkContext.parallelize(recordList)

### 문제 3-1: 이름으로 합계를 구해보자. 올바른 출력은 다음과 같다.

```

'임하나' 170.0

'김하나' 180.0

'김갑돌' 180.8

```

In [3]:
name_and_sum = recordRdd.map(lambda x: (x[0], x[1][1]))\
                            .reduceByKey(lambda x,y:x+y)

print(name_and_sum.collect())

[('김하나', 180), ('임하나', 170), ('김갑돌', 180.8)]


### 문제 3-2 : 과목으로 합계를 계산해 보자. 출력은 다음과 같이 나와야 한다.

```

'English' 252.3

'Math' 278.5

```

In [4]:
subject_and_sum = recordRdd.map(lambda x: (x[1][0], x[1][1]))\
                            .reduceByKey(lambda x,y:x+y)
print(subject_and_sum.collect())

[('English', 252.3), ('Math', 278.5)]


### 문제 3-3: 이름으로 합계과 개수를 구해보자. 출력은 다음과 같이 계산된다.

```

'임하나' (170.0, 2)

'김하나' (180.0, 2)

'김갑돌' (180.8, 2)

```

In [5]:
name_and_sumCount = recordRdd.map(lambda x: (x[0], x[1][1]))\
                                .combineByKey(lambda value: (value,1),
                                     lambda x,value: (x[0]+value, x[1]+1),
                                     lambda x,y: (x[0]+y[0], x[1]+y[1]))
print(name_and_sumCount.collect())

[('김하나', (180, 2)), ('임하나', (170, 2)), ('김갑돌', (180.8, 2))]


### 문제 3-4: 이름으로 평균을 계산해 보자. 앞서 3-3에서 사용했던 결과를 활용하고, 올바른 출력은 다음과 같다.

```

'임하나' 85.0

'김하나' 90.0

'김갑돌' 90.4

```

In [6]:
name_and_average = name_and_sumCount.map(lambda x:(x[0],x[1][0]/x[1][1]))

print(name_and_average.collect())

[('김하나', 90.0), ('임하나', 85.0), ('김갑돌', 90.4)]


In [7]:
spark.stop()